In [5]:
from keras.optimizers import Adam
import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
import random
from keras import backend as K
smooth = 1
import cv2
import os
import pickle
from keras.engine.topology import Input
from keras.layers import concatenate,Conv2DTranspose
if not os.path.exists('pickled'):
    os.makedirs('pickled')


In [6]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


data = []
labels = []
if not os.path.exists('pickled/seg_data.pkl'):
    directory = os.fsencode("melanoma")
    for file in os.listdir(directory):
        filename = os.fsdecode(file) 
        
        path = str('melanoma\\' + filename)
        image = cv2.imread(path)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        data.append(image)
        
        path = str('gt\\' + filename.split('.')[0] + '_segmentation.png')
        
        image = cv2.imread(path)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        labels.append(image)

    directory = os.fsencode("others")
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        path = str('others\\' + filename)
        image = cv2.imread(path)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        data.append(image)
        
        path = str('gt\\' + filename.split('.')[0] + '_segmentation.png')
        
        image = cv2.imread(path)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        labels.append(image)
        
    d = {}
    d['data'] = data
    d['labels'] = labels
    pickle.dump(d, open('pickled/seg_data.pkl', 'wb'))
else:
    d = pickle.load(open( 'pickled/seg_data.pkl', 'rb'))
    data = d['data']
    labels = d['labels']
                

In [7]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels, dtype="float") / 255.0

(x_train, x_test, y_train, y_test) = train_test_split(data,labels, test_size=0.20, random_state=42)


In [4]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


batch_size = 32
num_classes = 2
epochs = 10

img_rows, img_cols = 64, 64

input_shape = (img_rows, img_cols,3)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

inputs = Input((img_rows, img_cols, 3))

conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

conv10 = Conv2D(3, (1, 1), activation='sigmoid')(conv9)

model = Model(inputs=[inputs], outputs=[conv10])

model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

#model.compile(loss='binary_crossentropy',optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Score:', score[1])

x_train shape: (1600, 64, 64, 3)
1600 train samples
400 test samples
Train on 1600 samples, validate on 400 samples
Epoch 1/10
1600/1600 [==============================] - 454s 284ms/step - loss: -0.2766 - dice_coef: 0.2766 - val_loss: -0.2863 - val_dice_coef: 0.2863
Epoch 2/10
1600/1600 [==============================] - 426s 266ms/step - loss: -0.2769 - dice_coef: 0.2769 - val_loss: -0.2872 - val_dice_coef: 0.2872
Epoch 3/10
1600/1600 [==============================] - 443s 277ms/step - loss: -0.2784 - dice_coef: 0.2784 - val_loss: -0.2890 - val_dice_coef: 0.2890
Epoch 4/10
1600/1600 [==============================] - 415s 259ms/step - loss: -0.2825 - dice_coef: 0.2825 - val_loss: -0.3009 - val_dice_coef: 0.3009
Epoch 5/10
1600/1600 [==============================] - 475s 297ms/step - loss: -0.3281 - dice_coef: 0.3281 - val_loss: -0.3691 - val_dice_coef: 0.3691
Epoch 6/10
1600/1600 [==============================] - 434s 271ms/step - loss: -0.3889 - dice_coef: 0.3889 - val_loss: -0.4

In [ ]:
cv2.imshow('f',y_test[0])
cv2.waitKey()